# Build a Chatbot

In [1]:
pip install langchain-core langgraph>0.2.27

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install -qU "langchain[groq]"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
os.environ['LANGSMITH_PROJECT'] = 'Build-a-Chatbot'

 ········


In [19]:
LANGSMITH_TRACING='true'
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY="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"
LANGSMITH_PROJECT="Build-a-Chatbot"

GROQ_API_KEY="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"

In [17]:
import getpass
import os

if not os.environ.get('GROQ_API_KEY'):
    os.environ['GROQ_API_KEY'] = getpass.getpass('Enter API key for Groq: ')

from langchain.chat_models import init_chat_model

model = init_chat_model('llama3-8b-8192', model_provider = 'groq')


In [21]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content = 'Hi! I am Ravi')])

AIMessage(content='Nice to meet you, Ravi! How are you doing today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 16, 'total_tokens': 31, 'completion_time': 0.0125, 'prompt_time': 0.002405912, 'queue_time': 0.23695191999999998, 'total_time': 0.014905912}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run-4ddee4cc-90c6-4ff8-8d0e-46caaa49d8e4-0', usage_metadata={'input_tokens': 16, 'output_tokens': 15, 'total_tokens': 31})

In [22]:
model.invoke([HumanMessage(content = "What's my name?")])

AIMessage(content="I'm happy to help! However, I'm a large language model, I don't have the ability to know your personal information, including your name. Each time you interact with me, it's a new conversation, and I don't retain any information from previous conversations. If you'd like to share your name with me, I'd be happy to learn it, but it's not necessary for our conversation.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 15, 'total_tokens': 99, 'completion_time': 0.07, 'prompt_time': 0.004425841, 'queue_time': 0.346869594, 'total_time': 0.074425841}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-8ab09b36-98aa-4f5e-869a-e1d60c5441ce-0', usage_metadata={'input_tokens': 15, 'output_tokens': 84, 'total_tokens': 99})

### Passing the entire conversation history into the model

In [23]:
from langchain_core.messages import AIMessage

model.invoke(
    [
     HumanMessage(content = "Hi! I'm Ravi"),
     AIMessage(content="Hello Ravi! How can I assist you today?"),
     HumanMessage(content = "What's my name?"),
    ]
)

AIMessage(content='Your name is Ravi!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 42, 'total_tokens': 49, 'completion_time': 0.005833333, 'prompt_time': 0.005585333, 'queue_time': 0.233984297, 'total_time': 0.011418666}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-fac5286a-cee9-483d-903a-6341f96a3455-0', usage_metadata={'input_tokens': 42, 'output_tokens': 7, 'total_tokens': 49})

## Message Persistence

In [24]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema = MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer = memory)


### Creating a config

In [25]:
config = {"configurable": {"thread_id": "abc123"}}

In [26]:
query = "Haii, I'm Ravi."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Nice to meet you, Ravi!


In [27]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I remember! Your name is Ravi!


In [28]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm happy to help! However, I'm a large language model, I don't have any information about your personal identity, including your name. Each time you interact with me, it's a new conversation and I don't retain any information about you. So, I don't actually know your name!


#### Going back to the Original Conversation

In [29]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Ravi!


## Prompt templates

In [33]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

### Now we updating our application to incorporate this template

In [34]:
workflow = StateGraph(state_schema = MessagesState)

def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [35]:
# Invoking the application
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


================================ Human Message =================================

Hi! I'm Jim.


In [36]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


================================ Human Message =================================

What is my name?


In [37]:
# Making prompt complicated
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",        
        ),
        MessagesPlaceholder(variable_name = "messages"),
    ]
)

### Updating the State

In [44]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

workflow = StateGraph(state_schema = State)

def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [45]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hola Bob! ¡Encantado de conocerte! (Hello Bob! Nice to meet you!) ¿En qué puedo ayudarte hoy? (What can I help you with today?)


In [46]:
query = "What is name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Me llamo Asistente, soy un asistente virtual que está aquí para ayudarte con tus preguntas y necesidades. No tengo un nombre personal, pero puedo ayudarte con cualquier cosa que necesites. ¡Estoy aquí para ayudarte!


## Managing Conversation History